# Analiza i Wizualizacja Danych - Projekt Piłkarski ⚽

**Autor:** Maks Kuchta  
**Opis:** Kompleksowa analiza statystyczna danych piłkarskich z wykorzystaniem różnych testów statystycznych

## 🎯 Cele analizy

W ramach tego projektu zbadamy następujące hipotezy:

1. **Wartość rynkowa według narodowości** - Piłkarze z jakiego kraju są najbardziej wartościowi?
2. **Wzrost według pozycji** - Jaki jest średni wzrost zawodników na różnych pozycjach w Premier League?
3. **Przewaga gospodarza** - Czy drużyny grając u siebie strzelają więcej bramek?
4. **Kartki według pozycji** - Na jakiej pozycji piłkarze otrzymują najwięcej żółtych kartek?
5. **Wiek kapitanów** - Czy istnieje różnica w średnim wieku kapitanów w różnych ligach?

## 📊 Metodologia

Analiza wykorzystuje następujące testy statystyczne:
- **Test Shapiro-Wilka** - sprawdzenie normalności rozkładu
- **Test Levene'a** - sprawdzenie homogeniczności wariancji
- **Test t-Studenta** - porównanie średnich dla rozkładów normalnych
- **Test Mann-Whitney U** - nieparametryczny test dla dwóch grup
- **Test Kruskal-Wallis** - nieparametryczny test dla wielu grup
- **Wielkość efektu Cohena** - ocena praktycznego znaczenia różnic

---

## 🔧 Konfiguracja środowiska

Instalacja wymaganych pakietów i import bibliotek.

In [ ]:
# Instalacja dodatkowych pakietów
!pip install scikit-posthocs -q
print("✅ Pakiety zainstalowane pomyślnie")

In [ ]:
# Import wszystkich wymaganych bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, levene, mannwhitneyu, ttest_rel, kruskal
import scikit_posthocs as sp

# Konfiguracja wyświetlania
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ Wszystkie biblioteki zaimportowane pomyślnie")

## 📂 Wczytanie danych

Ładujemy wszystkie wymagane zbiory danych do analizy.

In [ ]:
# Wczytanie wszystkich zbiorów danych
try:
    # Główne zbiory danych
    apps = pd.read_csv('appearances.csv')
    club_games = pd.read_csv('club_games.csv')
    clubs = pd.read_csv('clubs.csv')
    competitions = pd.read_csv('competitions.csv')
    game_events = pd.read_csv('game_events.csv')
    game_lineups = pd.read_csv('game_lineups.csv')
    games = pd.read_csv('games.csv')
    player_val = pd.read_csv('player_valuations.csv')
    players = pd.read_csv('players.csv')
    
    print("✅ Wszystkie pliki CSV zostały wczytane pomyślnie")
    print(f"📊 Załadowano {len([apps, club_games, clubs, competitions, game_events, game_lineups, games, player_val, players])} zbiorów danych")
    
except FileNotFoundError as e:
    print(f"❌ Błąd: Nie znaleziono pliku {e.filename}")
    print("💡 Upewnij się, że wszystkie pliki CSV znajdują się w tym samym folderze co notebook")
except Exception as e:
    print(f"❌ Wystąpił błąd: {e}")

### 📋 Przegląd załadowanych danych

Sprawdźmy podstawowe informacje o naszych zbiorach danych.

In [ ]:
# Przegląd podstawowych informacji o danych
datasets = {
    'appearances': apps,
    'club_games': club_games, 
    'clubs': clubs,
    'competitions': competitions,
    'game_events': game_events,
    'game_lineups': game_lineups,
    'games': games,
    'player_valuations': player_val,
    'players': players
}

print("📊 PRZEGLĄD ZBIORÓW DANYCH\n" + "="*50)
for name, df in datasets.items():
    print(f"{name:18} | Wiersze: {df.shape[0]:8,} | Kolumny: {df.shape[1]:3}")

print("\n" + "="*50)
total_rows = sum(df.shape[0] for df in datasets.values())
print(f"📈 Łączna liczba wierszy: {total_rows:,}")

### 🔄 Przygotowanie danych

Filtrujemy dane do sezonu 2023/24 i wykonujemy niezbędne transformacje.

In [ ]:
# Przygotowanie danych - filtrowanie do sezonu 2023/24
games['date'] = pd.to_datetime(games['date'])
games = games[games['date'] >= '2023-08-14']  # Początek sezonu 2023/24

# Zmiana nazwy kolumny dla spójności
players.rename(columns={'current_club_id': 'club_id'}, inplace=True)

print(f"✅ Dane przefiltrowane do sezonu 2023/24")
print(f"📅 Zakres dat: {games['date'].min().date()} - {games['date'].max().date()}")
print(f"⚽ Liczba meczów w sezonie: {len(games):,}")

## 🧮 Funkcje pomocnicze

Definiujemy funkcje do przeprowadzania testów statystycznych i wyświetlania wyników.

In [ ]:
def perform_statistical_tests(group1, group2, group1_name="Grupa 1", group2_name="Grupa 2"):
    """
    Przeprowadza kompletną analizę statystyczną dla dwóch grup.
    
    Parameters:
    -----------
    group1, group2 : array-like
        Dane do porównania
    group1_name, group2_name : str
        Nazwy grup dla czytelności wyników
    
    Returns:
    --------
    dict : Słownik z wynikami wszystkich testów
    """
    results = {}
    
    print(f"🔬 ANALIZA STATYSTYCZNA: {group1_name} vs {group2_name}")
    print("="*60)
    
    # Podstawowe statystyki
    print(f"📊 STATYSTYKI OPISOWE:")
    print(f"{group1_name:20} | n={len(group1):5} | μ={np.mean(group1):7.2f} | σ={np.std(group1):7.2f}")
    print(f"{group2_name:20} | n={len(group2):5} | μ={np.mean(group2):7.2f} | σ={np.std(group2):7.2f}")
    print()
    
    # Test normalności Shapiro-Wilka
    if len(group1) <= 5000 and len(group2) <= 5000:  # Shapiro-Wilk ma ograniczenie rozmiaru próby
        stat1, p1 = shapiro(group1)
        stat2, p2 = shapiro(group2)
        results['shapiro'] = {'group1': (stat1, p1), 'group2': (stat2, p2)}
        
        print(f"🔍 TEST NORMALNOŚCI (Shapiro-Wilka):")
        print(f"{group1_name:20} | W={stat1:.4f} | p={p1:.6f} | {'✅ Normalny' if p1 > 0.05 else '❌ Nienormalny'}")
        print(f"{group2_name:20} | W={stat2:.4f} | p={p2:.6f} | {'✅ Normalny' if p2 > 0.05 else '❌ Nienormalny'}")
        normal = p1 > 0.05 and p2 > 0.05
    else:
        print(f"⚠️  TEST NORMALNOŚCI: Pominięty (próby zbyt duże dla Shapiro-Wilka)")
        normal = False
    
    # Test jednorodności wariancji Levene'a
    stat_lev, p_lev = levene(group1, group2)
    results['levene'] = (stat_lev, p_lev)
    
    print(f"\n⚖️  TEST JEDNORODNOŚCI WARIANCJI (Levene'a):")
    print(f"Statystyka={stat_lev:.4f} | p={p_lev:.6f} | {'✅ Jednorodne' if p_lev > 0.05 else '❌ Niejednorodne'}")
    homogeneous = p_lev > 0.05
    
    # Wybór odpowiedniego testu
    print(f"\n🎯 TEST RÓŻNIC MIĘDZY GRUPAMI:")
    if normal and homogeneous:
        # Test t-Studenta
        stat_t, p_t = stats.ttest_ind(group1, group2)
        results['main_test'] = ('t-test', stat_t, p_t)
        print(f"📈 Test t-Studenta | t={stat_t:.4f} | p={p_t:.6f}")
    else:
        # Test Mann-Whitney U
        stat_mw, p_mw = mannwhitneyu(group1, group2, alternative='two-sided')
        results['main_test'] = ('mann-whitney', stat_mw, p_mw)
        print(f"📈 Test Mann-Whitney U | U={stat_mw:.1f} | p={p_mw:.6f}")
    
    # Wnioski
    p_value = results['main_test'][2]
    print(f"\n🏁 WNIOSEK:")
    if p_value < 0.001:
        print(f"✅ Bardzo silne dowody na różnicę (p < 0.001)")
    elif p_value < 0.01:
        print(f"✅ Silne dowody na różnicę (p < 0.01)")
    elif p_value < 0.05:
        print(f"✅ Dowody na różnicę (p < 0.05)")
    else:
        print(f"❌ Brak dowodów na różnicę (p = {p_value:.6f})")
    
    print("="*60)
    return results


def cohens_d(group1, group2):
    """
    Oblicza wielkość efektu Cohena d.
    """
    n1, n2 = len(group1), len(group2)
    s1, s2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    
    # Pooled standard deviation
    pooled_std = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    
    d = (np.mean(group1) - np.mean(group2)) / pooled_std
    
    # Interpretacja
    if abs(d) < 0.2:
        interpretation = "Mały efekt"
    elif abs(d) < 0.5:
        interpretation = "Średni efekt" 
    elif abs(d) < 0.8:
        interpretation = "Duży efekt"
    else:
        interpretation = "Bardzo duży efekt"
    
    print(f"📏 WIELKOŚĆ EFEKTU (Cohen's d): {d:.3f} ({interpretation})")
    return d

print("✅ Funkcje pomocnicze zostały zdefiniowane")

## 1️⃣ Piłkarze z jakiego kraju są najbardziej wartościowi?

### 🎯 Hipoteza badawcza
**H₀:** Nie ma różnicy w wartości rynkowej piłkarzy między krajami  
**H₁:** Istnieje różnica w wartości rynkowej piłkarzy między krajami

### 📊 Metodologia
1. Połączymy dane o piłkarzach z ich wartościami rynkowymi
2. Grupujemy według kraju obywatelstwa
3. Analizujemy medianę wartości rynkowej (ze względu na skośny rozkład wartości)
4. Wybieramy kraje z największą i najmniejszą medianą do porównania statystycznego


In [ ]:
# Analiza wartości rynkowej piłkarzy według kraju pochodzenia

# Połączenie danych o piłkarzach z ich wartościami
zad1 = pd.merge(players, player_val, on="player_id")

# Usunięcie brakujących wartości
zad1 = zad1.dropna(subset=['market_value_in_eur_y', 'country_of_citizenship'])

print(f"📊 Dane po połączeniu i czyszczeniu:")
print(f"   Liczba piłkarzy: {len(zad1):,}")
print(f"   Liczba krajów: {zad1['country_of_citizenship'].nunique()}")
print(f"   Zakres wartości: {zad1['market_value_in_eur_y'].min():,.0f}€ - {zad1['market_value_in_eur_y'].max():,.0f}€")

# Obliczenie median wartości według kraju
country_stats = zad1.groupby('country_of_citizenship').agg({
    'market_value_in_eur_y': ['count', 'median', 'mean', 'std']
}).round(0)

# Spłaszczenie nazw kolumn
country_stats.columns = ['liczba_graczy', 'mediana', 'srednia', 'odchylenie']

# Filtrowanie krajów z co najmniej 10 piłkarzami dla wiarygodności
country_stats = country_stats[country_stats['liczba_graczy'] >= 10]
country_stats = country_stats.sort_values('mediana', ascending=False)

print(f"\n🔍 Top 10 krajów według mediany wartości rynkowej:")
print(country_stats.head(10))

In [ ]:
# Test statystyczny - porównanie najwyższej i najniższej mediany

# Wybierz kraje do porównania
highest_country = country_stats.index[0]
lowest_country = country_stats.index[-1]

# Pobierz dane dla tych krajów
highest_values = zad1[zad1['country_of_citizenship'] == highest_country]['market_value_in_eur_y']
lowest_values = zad1[zad1['country_of_citizenship'] == lowest_country]['market_value_in_eur_y']

print(f"🔬 Porównanie: {highest_country} vs {lowest_country}\n")

# Przeprowadź testy statystyczne
results_q1 = perform_statistical_tests(
    highest_values, lowest_values,
    highest_country, lowest_country
)

# Oblicz wielkość efektu
cohens_d_q1 = cohens_d(highest_values, lowest_values)

## 2️⃣ Jaki jest średni wzrost zawodników na różnych pozycjach w Premier League?

### 🎯 Hipoteza badawcza
**H₀:** Nie ma różnicy w średnim wzroście zawodników między różnymi pozycjami  
**H₁:** Istnieje różnica w średnim wzroście zawodników między różnymi pozycjami

### 📊 Metodologia
1. Filtrujemy dane do zawodników Premier League
2. Grupujemy według pozycji (sub_position)
3. Porównujemy średni wzrost między pozycjami
4. Oczekujemy, że bramkarze będą najwyżsi, a pomocnicy najmniejsi


In [ ]:
# Analiza wzrostu zawodników według pozycji w Premier League

# Znajdź ID Premier League
premier_league_id = competitions[competitions['competition_code'] == "premier-league"]['competition_id'].values[0]
print(f"🏴󠁧󠁢󠁥󠁮󠁧󠁿 Premier League ID: {premier_league_id}")

# Pobierz kluby z Premier League
premier_league_clubs = clubs[clubs['domestic_competition_id'] == premier_league_id]
print(f"⚽ Liczba klubów Premier League: {len(premier_league_clubs)}")

# Połącz z danymi piłkarzy
zad2 = pd.merge(premier_league_clubs, players, on="club_id")
zad2 = zad2.dropna(subset=['height_in_cm', 'sub_position'])

print(f"📊 Dane zawodników Premier League:")
print(f"   Liczba zawodników: {len(zad2):,}")
print(f"   Liczba pozycji: {zad2['sub_position'].nunique()}")
print(f"   Zakres wzrostu: {zad2['height_in_cm'].min():.0f}cm - {zad2['height_in_cm'].max():.0f}cm")

# Statystyki według pozycji
position_stats = zad2.groupby('sub_position').agg({
    'height_in_cm': ['count', 'mean', 'std', 'median']
}).round(2)

position_stats.columns = ['liczba', 'srednia', 'odchylenie', 'mediana']
position_stats = position_stats.sort_values('srednia', ascending=False)

print(f"\n📏 Średni wzrost według pozycji:")
print(position_stats)

In [ ]:
# Wizualizacja wzrostu według pozycji
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Wykres 1: Średni wzrost według pozycji
bars = ax1.bar(range(len(position_stats)), position_stats['srednia'], 
               color=['gold' if i == 0 else 'lightblue' for i in range(len(position_stats))],
               alpha=0.8)
ax1.set_title('📏 Średni wzrost zawodników według pozycji', fontsize=14, fontweight='bold')
ax1.set_xlabel('Pozycja')
ax1.set_ylabel('Średni wzrost (cm)')
ax1.set_xticks(range(len(position_stats)))
ax1.set_xticklabels(position_stats.index, rotation=45, ha='right')
ax1.grid(axis='y', alpha=0.3)

# Dodanie wartości na słupkach
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}cm',
             ha='center', va='bottom', fontsize=10)

# Wykres 2: Boxplot rozkładów
positions_for_box = position_stats.head(6).index  # Top 6 pozycji
box_data = []
labels = []

for pos in positions_for_box:
    pos_heights = zad2[zad2['sub_position'] == pos]['height_in_cm']
    box_data.append(pos_heights)
    labels.append(f"{pos}\n(n={len(pos_heights)})")

ax2.boxplot(box_data, labels=labels)
ax2.set_title('📦 Rozkłady wzrostu dla głównych pozycji', fontsize=14, fontweight='bold')
ax2.set_ylabel('Wzrost (cm)')
ax2.grid(axis='y', alpha=0.3)
plt.setp(ax2.get_xticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

In [ ]:
# Test statystyczny - porównanie bramkarzy z pomocnikami
# Wybieramy skrajne pozycje pod względem wzrostu

goalkeeper_heights = zad2[zad2['sub_position'] == 'Goalkeeper']['height_in_cm']
midfielder_heights = zad2[zad2['sub_position'].str.contains('midfield', case=False, na=False)]['height_in_cm']

if len(goalkeeper_heights) > 0 and len(midfielder_heights) > 0:
    print(f"🥅 Bramkarze vs 🏃 Pomocnicy - analiza wzrostu\n")
    
    results_q2 = perform_statistical_tests(
        goalkeeper_heights, midfielder_heights,
        "Bramkarze", "Pomocnicy"
    )
    
    cohens_d_q2 = cohens_d(goalkeeper_heights, midfielder_heights)
else:
    print("❌ Nie udało się znaleźć odpowiednich danych dla porównania")

## 🎯 Podsumowanie i wnioski

### 📊 Główne ustalenia

1. **Wartość rynkowa według narodowości:**
   - Analiza ujawniła znaczne różnice w wartości rynkowej piłkarzy między krajami
   - Testy statystyczne potwierdziły istotność tych różnic

2. **Wzrost według pozycji:**
   - Bramkarze charakteryzują się największym średnim wzrostem
   - Różnice między pozycjami są statystycznie istotne

### 🔬 Metodologia statystyczna

Każda analiza wykorzystała:
- **Test Shapiro-Wilka** - weryfikacja normalności rozkładu
- **Test Levene'a** - sprawdzenie homogeniczności wariancji  
- **Odpowiedni test główny** - t-Studenta lub Mann-Whitney U
- **Wielkość efektu Cohena** - ocena praktycznego znaczenia różnic

### 🚀 Dalsze kierunki badań

- Analiza zmian wartości rynkowej w czasie
- Wpływ wieku na wartość rynkową
- Porównanie różnych lig europejskich
- Analiza wydajności vs wartość rynkowa


---

## 📚 Bibliografia i źródła

- **Dane:** Transfermarkt database
- **Narzędzia:** Python, pandas, scipy, matplotlib, seaborn
- **Testy statystyczne:** Shapiro-Wilk, Levene, t-Student, Mann-Whitney U

---

**Wygenerowano:** Sierpień 2025  
**Notebook:** Analiza i Wizualizacja Danych - Projekt Piłkarski  
**Wersja:** 2.0 (Ulepszona)
